In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
      #  print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import re
import html
import unicodedata
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.stem import WordNetLemmatizer

import lightgbm as lgb
import xgboost as xgb
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import  LogisticRegressionCV
import gc
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings("ignore")
pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 100)
import os
from sklearn.metrics import cohen_kappa_score

In [ ]:
def kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

In [ ]:
breeds = pd.read_csv('../input/petfinder-adoption-prediction/BreedLabels.csv')
colors = pd.read_csv('../input/petfinder-adoption-prediction/ColorLabels.csv')
states = pd.read_csv('../input/petfinder-adoption-prediction/state_labels.csv')

train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
test = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv', engine='python')
sub = pd.read_csv('../input/petfinder-adoption-prediction/test/sample_submission.csv')

In [ ]:
train_img_features=pd.read_csv("../input/train-img-features/train_img_features.csv")

In [ ]:
train_img_features.head()

In [ ]:
train["PetID"].head()

In [ ]:
test_img_features=pd.read_csv("../input/testimgfeatures/test_img_features.csv")

In [ ]:
test["PetID"].head()

In [ ]:
test_img_features.head()

In [ ]:
train_img_features.shape

In [ ]:
train_img_features.set_index("Unnamed: 0",inplace=True)

In [ ]:
test_img_features.set_index("Unnamed: 0",inplace=True)

In [ ]:
train_img_features.columns = ["img_feat{}".format(i) for i in range(256)]
test_img_features.columns = ["img_feat{}".format(i) for i in range(256)]

train_img_features["PetID"] = train_img_features.index
test_img_features["PetID"] = test_img_features.index

train = pd.merge(train, train_img_features, on="PetID")
test = pd.merge(test, test_img_features, on="PetID")

print(train.shape, test.shape)

In [ ]:
print(train.shape, test.shape)

# Find+clean nulls

In [ ]:

    ##

train.loc[:,"Description"] = train.loc[:,"Description"].fillna("MISSINGG")
    
#train_proc.dropna(axis=0,inplace=True)
test.loc[:,"Description"] = test.loc[:,"Description"].fillna("MISSINGG")

# #feature engineering

In [ ]:


train['Name'] = train['Name'].fillna('No_name_found')
test['Name'] = test['Name'].fillna('No_name_found')
#all_data['Name'] = all_data['Name'].fillna('No_name_found')

train['No_name'] = 0
train.loc[train['Name'] == 'No_name_found', 'No_name'] = 1
test['No_name'] = 0
test.loc[test['Name'] == 'No_name_found', 'No_name'] = 1
#all_data['No_name'] = 0
#all_data.loc[all_data['Name'] == 'No_name_found', 'No_name'] = 1


train['Pure_breed'] = 0
train.loc[train['Breed2'] == 0, 'Pure_breed'] = 1

test['Pure_breed'] = 0
test.loc[test['Breed2'] == 0, 'Pure_breed'] = 1

train["desc_length"]=train["Description"].apply(lambda x: len(x.split()))
test["desc_length"]=test["Description"].apply(lambda x: len(x.split()))
#all_data['Pure_breed'] = 0
#all_data.loc[all_data['Breed2'] == 0, 'Pure_breed'] = 1

In [ ]:
train["desc_length"][8]

In [ ]:
train["Description"][8]

In [ ]:
print(train.shape, test.shape)

In [ ]:
train['dataset_type'] = 'train'
test['dataset_type'] = 'test'
all_data = pd.concat([train, test])

In [ ]:
print(train.shape, test.shape)

# Parsing sentiment data

information about Google's  NLP API:
https://cloud.google.com/natural-language/docs/basics#sentiment-analysis-values

The score of a document's sentiment indicates the overall emotion of a document. The magnitude of a document's sentiment indicates how much emotional content is present within the document, and this value is often proportional to the length of the document.v

A document with a neutral score (around 0.0) may indicate a low-emotion document, or may indicate mixed emotions, with both high positive and negative values which cancel each out. Generally, you can use magnitude values to disambiguate these cases, as truly neutral documents will have a low magnitude value, while mixed documents will have higher magnitude values.

In [ ]:
#This code was  inspired from this amazing repo:
#https://github.com/liyi61/PetFinder.my-Adoption-Speed-Prediction/tree/master
sentiment_dict={}
for filename in os.listdir("../input/petfinder-adoption-prediction/train_sentiment/"):
    with open("../input/petfinder-adoption-prediction/train_sentiment/"+filename, 'rb') as f:
        sentiment = json.load(f)
    pet_id=filename.split('.')[0]
    sentiment_dict[pet_id]={}
    sentiment_dict[pet_id]["magnitude"]=sentiment["documentSentiment"]["magnitude"]
    sentiment_dict[pet_id]['score'] = sentiment['documentSentiment']['score']
    sentiment_dict[pet_id]['language'] = sentiment['language']
#

for filename in os.listdir('../input/petfinder-adoption-prediction/test_sentiment/'):
    with open('../input/petfinder-adoption-prediction/test_sentiment/' + filename, 'rb') as f:
        sentiment = json.load(f)
    pet_id = filename.split('.')[0]
    sentiment_dict[pet_id] = {}
    sentiment_dict[pet_id]['magnitude'] = sentiment['documentSentiment']['magnitude']
    sentiment_dict[pet_id]['score'] = sentiment['documentSentiment']['score']
    sentiment_dict[pet_id]['language'] = sentiment['language']


In [ ]:
train['lang'] = train['PetID'].apply(lambda x: sentiment_dict[x]['language'] if x in sentiment_dict else 'no')
train['magnitude'] = train['PetID'].apply(lambda x: sentiment_dict[x]['magnitude'] if x in sentiment_dict else 0)
train['score'] = train['PetID'].apply(lambda x: sentiment_dict[x]['score'] if x in sentiment_dict else 0)

test['lang'] = test['PetID'].apply(lambda x: sentiment_dict[x]['language'] if x in sentiment_dict else 'no')
test['magnitude'] = test['PetID'].apply(lambda x: sentiment_dict[x]['magnitude'] if x in sentiment_dict else 0)
test['score'] = test['PetID'].apply(lambda x: sentiment_dict[x]['score'] if x in sentiment_dict else 0)

all_data['lang'] = all_data['PetID'].apply(lambda x: sentiment_dict[x]['language'] if x in sentiment_dict else 'no')
all_data['magnitude'] = all_data['PetID'].apply(lambda x: sentiment_dict[x]['magnitude'] if x in sentiment_dict else 0)
all_data['score'] = all_data['PetID'].apply(lambda x: sentiment_dict[x]['score'] if x in sentiment_dict else 0)

In [ ]:
print(train.shape, test.shape)

# Parsing img metadata

Let's take an idea about what the vision API does;

1.**labelAnnotations**: Labels can identify general objects, locations, activities, animal species, products, and more. 
A LABEL_DETECTION response includes the detected labels, their score, topicality, and an opaque label ID, where:
(https://cloud.google.com/vision/docs/labels)
       mid - if present, contains a machine-generated identifier (MID) corresponding to the entity's Google Knowledge Graph entry. Note that mid values remain unique across different languages, so you can use these values to tie entities together from different languages. To inspect MID values, refer to the Google Knowledge Graph API documentation.
   
    description - the label description.
    
    score - the confidence score, which ranges from 0 (no confidence) to 1 (very high confidence).
    (scores are stored from the highest to the lowest. therefore, I'll only consider the **1st description** for each image. This explains why we're indexing ["labelAnnotations"][score] **[0]**
    topicality - The relevancy of the ICA (Image Content Annotation) label to the image. It measures how important/central a label is to the overall context of a page.


2.**imagePropertiesAnnotation**(https://cloud.google.com/vision/docs/detecting-properties)

The Image Properties feature detects general attributes of the image, such as dominant color.

a.dominant color>>colors>>scores(https://github.com/googleapis/google-cloud-node/issues/1324)
:, higher "scores" means higher confidence that the color in question is prominent in the central focus of the image,
again,sorted from the highest to  lowest.

b.dominant color>>colors>>pixelFraction 	Stores the fraction of pixels the color occupies in the image. Value in range [0, 1]. 

3.**cropHintsAnnotation**
cropHints>>boundingPoly: vertices for a bounding polygon for the detected image annotation. 
cropHints>>ImportanceFraction: Fraction of importance of this salient(polygon) region with respect to the original image.

In [ ]:
train["lang"].value_counts()#we can drop this column since english dominates other languages

In [ ]:
train.info()

In [ ]:
cols_to_use = ['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID', 'score',
       'VideoAmt', "Description", 'PhotoAmt', 'AdoptionSpeed', 'magnitude']
#feature engineering; 'health', 'Free',, 'No_name', 'Pure_breed', 'desc_length','desc_words',
          #     'average_word_length'

In [ ]:
len(cols_to_use)

In [ ]:
# train = train[[col for col in cols_to_use if col in train.columns]]
# test = test[[col for col in cols_to_use if col in test.columns]]

In [ ]:
train.info()

In [ ]:
cat_cols = ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'State', 'RescuerID',"lang"]
#Fee,Age,'VideoAmt', 'PhotoAmt'

In [ ]:
# more_cols = []
# #feature interaction
# for col1 in cat_cols:
#     for col2 in cat_cols:
#         if col1 != col2 and col1 not in ['RescuerID', 'State'] and col2 not in ['RescuerID', 'State']:
#             train[col1 + '_' + col2] = train[col1].astype(str) + '_' + train[col2].astype(str)
#             test[col1 + '_' + col2] = test[col1].astype(str) + '_' + test[col2].astype(str)
#             more_cols.append(col1 + '_' + col2)

In [ ]:
#cat_cols = cat_cols + more_cols

In [ ]:
indexer = {}
for col in cat_cols:
    # print(col)
    _, indexer[col] = pd.factorize(train[col].astype(str))
  #  train.loc[:,col]=pd.factorize(train.loc[:,col])[0]

In [ ]:
print(train.shape, test.shape)

In [ ]:
for col in cat_cols:
    # print(col)
    train[col] = indexer[col].get_indexer(train[col].astype(str))
    test[col] = indexer[col].get_indexer(test[col].astype(str))

In [ ]:
print(train.shape, test.shape)

In [ ]:
train.info()

In [ ]:
y_train = train['AdoptionSpeed']
train = train.drop(['AdoptionSpeed'], axis=1)

In [ ]:

from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train, y_train, test_size = 0.2,random_state=42)

In [ ]:
type(y_train)

In [ ]:
train_data =x_train.to_numpy()

In [ ]:
train_label0 = y_train.to_numpy()

In [ ]:
train_label0.shape[0]

In [ ]:

train_label=np.zeros([train_label0.shape[0],5])

In [ ]:
#convertLabels 2 1-hotEncoded
for i in range(train_label0.shape[0]):
    if train_label0[i] == 0:
        train_label[i, 0] = 1
    elif train_label0[i] == 1:
        train_label[i, 1] = 1
    elif train_label0[i] == 2:
        train_label[i, 2] = 1
    elif train_label0[i] == 3:
        train_label[i, 3] = 1
    elif train_label0[i] == 4:
        train_label[i, 4] = 1

In [ ]:
train_label[0]

In [ ]:

test_data = x_valid.to_numpy()

In [ ]:
test_label0 = y_valid.to_numpy()

# Model parameters

In [ ]:
num_examples_per_epoch_for_train = train_data.shape[0]
training_epochs = 70
batch_size = 1200
display_step = 1
LR = 0.0001

In [ ]:
import tensorflow as tf

In [ ]:
# Normalization
def norm(inputs, on_train):
    conv_mean, conv_var = tf.nn.moments(inputs, [0, 1, 2])

    scale = tf.Variable(tf.ones([inputs.shape[-1]]))
    shift = tf.Variable(tf.zeros([inputs.shape[-1]]))
    epsilon = 0.001
    ema = tf.train.ExponentialMovingAverage(decay=0.5)

    def mean_var_with_update():
        ema_apply_op = ema.apply([conv_mean, conv_var])
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(conv_mean), tf.identity(conv_var)

    mean, var = tf.cond(on_train, mean_var_with_update, lambda: (ema.average(conv_mean), ema.average(conv_var)))
    norm_out = tf.nn.batch_normalization(inputs, mean, var, shift, scale, epsilon)
    return norm_out

In [ ]:
# Random Drop out nodes to avoid overfitting 
def Dropout(x, rate):
    with tf.name_scope('Dropout_layer'):
        return tf.nn.dropout(x, rate)

In [ ]:
# Define Convolutional Layers
def add_layer(input, W_shape, b_shape, strides,  on_train,  layer_name, padding='SAME', activation_function=tf.nn.tanh):
    with tf.name_scope(layer_name):
        with tf.name_scope('Weights'):
        # Initialize weights
            Weight = tf.Variable(tf.truncated_normal(shape=W_shape, stddev=0.1, dtype=tf.float32))
        with tf.name_scope('Biases'):
        
            b = tf.Variable(tf.constant(0.01, shape=b_shape))
        with tf.name_scope('Conv'):
        
            conv_out = tf.nn.conv2d(input, Weight, strides, padding)
            conv_out_plus_b = tf.nn.bias_add(conv_out, b)
     
        with tf.name_scope('Activation'):
        #  activation, add non linear relationship
            x = activation_function(conv_out_plus_b)
    return x

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

In [ ]:
# Ddfine input placeholder 
with tf.name_scope('Inputs'):
    x_placeholder = tf.placeholder(tf.float32, [None, 294])
    x = tf.reshape(x_placeholder, [-1, 294, 1, 1])
    y = tf.placeholder(tf.int32, [None, 5])
    on_train = tf.placeholder(tf.bool)
    rate_place = tf.placeholder(tf.float32)

In [ ]:

# convolutioanl layer 1 
conv1 = add_layer(x, [3, 1, 1, 16], [16], [1, 1, 1, 1], on_train, 'Conv1_1')

In [ ]:
# ResNet Convolutional Block 1 (including two 3*1 convolutional layers)
with tf.name_scope('Res_block1'):
    conv1_2 = add_layer(conv1, [3, 1, 16, 16], [16], [1, 1, 1, 1], on_train, 'Conv1_2')
    conv1_3 = add_layer(conv1_2, [3, 1, 16, 16], [16], [1, 1, 1, 1], on_train, 'Conv1_3',
                        activation_function=tf.identity)
    fuse1 = tf.nn.tanh(tf.add(conv1, conv1_3))

trans1 = add_layer(fuse1, [3, 1, 16, 64], [64], [1, 1, 1, 1], on_train, 'Tans1')

# ResNet Convolutional Block 2 (including two 3*1 convolutional layers)
with tf.name_scope('Res_block2'):
    conv2_1 = add_layer(trans1, [3, 1, 64, 64], [64], [1, 1, 1, 1], on_train, 'Conv2_1')
    conv2_2 = add_layer(conv2_1, [3, 1, 64, 64], [64], [1, 1, 1, 1], on_train, 'Conv2_2',
                        activation_function=tf.identity)
    fuse2 = tf.nn.tanh(tf.add(trans1, conv2_2))

trans2 = add_layer(fuse2, [3, 1, 64, 128], [128], [1, 1, 1, 1], on_train, 'Tans2')

# ResNet Convolutional Block 3 (including two 3*1 convolutional layers)
with tf.name_scope('Res_block3'):
    conv3_1 = add_layer(trans2, [3, 1, 128, 128], [128], [1, 1, 1, 1], on_train, 'Conv3_1')
    conv3_2 = add_layer(conv3_1, [3, 1, 128, 128], [128], [1, 1, 1, 1], on_train, 'Conv3_2',
                        activation_function=tf.identity)
    fuse3 = tf.nn.tanh(tf.add(trans2, conv3_2))
    

In [ ]:
# vectorize features 37632
fuse_F_flat = tf.reshape(fuse3, [-1, 37632])

# Fully Connected 1
with tf.name_scope('FC1_layer'):
    with tf.name_scope('W_fc1'):
        W_fc1 = tf.Variable(tf.truncated_normal(shape=[37632, 1024], stddev=0.1, dtype=tf.float32))
    with tf.name_scope('b_fc1'):
        b_fc1 = tf.Variable(tf.constant(0.01, shape=[1024]))
    fc1 = tf.nn.tanh(tf.matmul(fuse_F_flat, W_fc1) + b_fc1)
    fc1_drop = Dropout(fc1, rate_place)
# Fully Connected 2
with tf.name_scope('FC1_layer'):
    with tf.name_scope('W_fc2'):
        W_fc2 = tf.Variable(tf.truncated_normal(shape=[1024, 1024], stddev=0.1, dtype=tf.float32))
    with tf.name_scope('b_fc2'):
        b_fc2 = tf.Variable(tf.constant(0.01, shape=[1024]))
    fc2 = tf.nn.tanh(tf.matmul(fc1_drop, W_fc2) + b_fc2)
    fc2_drop = Dropout(fc2, rate_place)
# Fully Connected 3
with tf.name_scope('FC1_layer'):
    with tf.name_scope('W_fc3'):
        W_fc3 = tf.Variable(tf.truncated_normal(shape=[1024, 512], stddev=0.1, dtype=tf.float32))
    with tf.name_scope('b_fc3'):
        b_fc3 = tf.Variable(tf.constant(0.01, shape=[512]))
    fc3 = tf.nn.tanh(tf.matmul(fc1_drop, W_fc3) + b_fc3)
    fc3_drop = Dropout(fc3, rate_place)

In [ ]:

# Prediction Output Layer with output dimendion 5
with tf.name_scope('Output'):
    with tf.name_scope('W_fc4'):
        W_fc4 = tf.Variable(tf.truncated_normal(shape=[512, 5], stddev=0.1, dtype=tf.float32))
    with tf.name_scope('b_fc4'):
        b_fc4 = tf.Variable(tf.constant(0.01, shape=[5]))
    y_pred = tf.matmul(fc3_drop, W_fc4) + b_fc4

In [ ]:
# Use Cross-entropy as loss function 
with tf.name_scope('Loss'):
    loss = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=y_pred)

with tf.name_scope('Train'):
    train_op = tf.train.AdamOptimizer(LR).minimize(loss)


In [ ]:
# To calculate accuracy
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [ ]:
GT = tf.argmax(y, 1)
PD = tf.argmax(y_pred, 1)


init_op = tf.global_variables_initializer()

# to save model
saver = tf.train.Saver(tf.global_variables())

# Start Session，run the network
with tf.Session() as sess:
    # write model and parameters into ckpt file 
    ckpt = tf.train.get_checkpoint_state('./model')
    
    sess.run(init_op)
    # number of batches
    total_batches = int(num_examples_per_epoch_for_train / batch_size)
    print('Per batch Size:', batch_size)
    print('Train sample Count Per Epoch:', num_examples_per_epoch_for_train)
    print('Total batch Count Per Epoch:', total_batches)
    training_step = 0
    # training 
    for epoch in range(training_epochs):
        for batch_idx in range(total_batches):
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size
            # training batch
            batch_xs = train_data[start_idx:end_idx, :]
            batch_ys = train_label[start_idx:end_idx, :]

            row_rand_array = np.arange(test_data.shape[0])
            np.random.shuffle(row_rand_array)
            batch_xs_val = test_data[row_rand_array[0:batch_size], :]
            batch_ys_val = test_label[row_rand_array[0:batch_size], :]

            # loss
            _, loss_ = sess.run([train_op, loss], {x_placeholder: batch_xs, y: batch_ys, on_train: True, rate_place: 0.7})
            training_step += 1
            
            if training_step % display_step == 0:
                
                result, gt = sess.run([PD, GT], feed_dict={x_placeholder: batch_xs, y: batch_ys, on_train: False, rate_place: 1})
                train_accuracy = kappa(gt, result)

                result_val, gt_val = sess.run([PD, GT], feed_dict={x_placeholder: batch_xs_val, y: batch_ys_val, rate_place: 1,
                    on_train: False})
                
                valid_accuracy = kappa(gt_val, result_val)

                # test
                print('Epoch:', epoch, '| Step:', training_step, '| train loss: %.4f' % loss_,
                      '| train accuracy: %.4f' % train_accuracy,  '| valid accuracy: %.4f' % valid_accuracy,)
        # save
        saver.save(sess=sess, save_path='./model/prediction.ckpt')

# TextCol(Descriptio)

## Text cleaning(Descriptio)

In [ ]:

def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))

def remove_html(text):
    html=re.compile(r"<.*?>")
    return  html.sub(r'',  text)
   # This should happen before all other preprocessing steps,
   # as we will see in the full pipeline, since it will help sentence and words tokenization for example, and will reduce vocab. 
   

def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


#7599:#Earthquake
#other   examples: earthquake, EARTHQUAKE

def to_lowercase(text):
    return text.lower()

#remove URLs
def remove_urls(text):
  #slower than regex
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def text2words(text):
    return word_tokenize(text)

def clean_text( text):
    text = remove_special_chars(text)
    text=remove_html(text)
    text=remove_urls(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    

  #  text = replace_numbers(text)
    words = text2words(text)
    #REMOVE STOPWORDS?
    words = remove_stopwords(words, stop_words)
    #words = stem_words(words)# Either stem ovocar lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

In [ ]:
descrip_clean=train["Description"].apply(lambda x:clean_text(x))

In [ ]:
descrip_clean_test=test["Description"].apply(lambda x:clean_text(x))

In [ ]:
train["Description_clean"]=descrip_clean

In [ ]:
test["Description_clean"]=descrip_clean_test

In [ ]:
print(train.shape, test.shape)

In [ ]:
train[train["Description_clean"]=="missingg"]

In [ ]:
train[train["lang"]!="en"]["Description"]

# encoding description

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
bow_tf = TfidfVectorizer( ngram_range=(1,2))

In [ ]:
#bow_tf_test = TfidfVectorizer( ngram_range=(1,2))

In [ ]:
x_text_tfidf = bow_tf.fit_transform(train["Description_clean"])##extTrial

In [ ]:
x_text_tfidf_test = bow_tf.fit_transform(test["Description_clean"])##extTrial

In [ ]:
x_text_tfidf

In [ ]:
from sklearn.decomposition import SparsePCA, TruncatedSVD, LatentDirichletAllocation, NMF

In [ ]:
text_features=[]
n_components = 5

svd_ = TruncatedSVD(
        n_components=n_components, random_state=1337)
svd_col = svd_.fit_transform(x_text_tfidf)
svd_col = pd.DataFrame(svd_col)
  # print(svd_col)
svd_col = svd_col.add_prefix('SVD_Description_')

#    nmf_col = nmf_.fit_transform(tfidf_col)
#    nmf_col = pd.DataFrame(nmf_col)
#    nmf_col = nmf_col.add_prefix('NMF_{}_'.format(i))

text_features.append(svd_col)

In [ ]:
# Combine all extracted features:
text_features = pd.concat(text_features, axis=1)
text_features.shape

In [ ]:
# Concatenate with main DF:
trai_svd=pd.concat([train, text_features], axis=1)
#test_svd=pd.concat([test, text_features], axis=1)
#trai_svd.head()

In [ ]:

#same4test
text_features=[]
n_components = 5

svd_ = TruncatedSVD(
        n_components=n_components, random_state=1337)
    #Non-Negative Matrix Factorization (NMF).
    #Find two non-negative matrices (W, H) whose product approximates the non- negative matrix X. 
#This factorization can be used for example for dimensionality reduction, source separation or topic extraction.
#     nmf_ = NMF(
#         n_components=n_components, random_state=1337)
svd_col = svd_.fit_transform(x_text_tfidf_test)
svd_col = pd.DataFrame(svd_col)
  # print(svd_col)
svd_col = svd_col.add_prefix('SVD_Description_')

#    nmf_col = nmf_.fit_transform(tfidf_col)
#    nmf_col = pd.DataFrame(nmf_col)
#    nmf_col = nmf_col.add_prefix('NMF_{}_'.format(i))

text_features.append(svd_col)
# Combine all extracted features:
text_features = pd.concat(text_features, axis=1)
text_features.shape
# Concatenate with main DF:
test_svd=pd.concat([test, text_features], axis=1)
test_svd.head()

In [ ]:
# test_svd=pd.concat([test, text_features], axis=1)
# test_svd.head()

In [ ]:
#descr_tf=pd.DataFrame(x_text_tfidf)

In [ ]:
#X_text=trai_svd["Description"]

trai_svd.drop(["Description", "Description_clean"], axis=1,inplace=True)

In [ ]:
#X_text=trai_svd["Description"]

test_svd.drop(["Description", "Description_clean"], axis=1,inplace=True)

In [ ]:
trai_svd.shape

In [ ]:
trai_svd.drop("PetID",axis=1,inplace=True)

In [ ]:
test_svd.drop("PetID",axis=1,inplace=True)

In [ ]:
#train.drop("Description", inplace=True,axis=1)

In [ ]:
#test.drop("Description", inplace=True,axis=1)

# train/test split

since dataset size is small,the validation_set won't be representative to estimate val_score on

# modeling

In [ ]:
n_fold = 5
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=15)

In [ ]:
folds

In [ ]:
trai_svd.drop(["Name", "dataset_type"],axis=1,inplace=True)

In [ ]:
test_svd.drop(["Name", "dataset_type"],axis=1,inplace=True)

In [ ]:
# trai_svd.drop(["lang"],axis=1,inplace=True)
# test_svd.drop(["lang"],axis=1,inplace=True)

In [ ]:
trai_svd.shape

In [ ]:
y.shape

In [ ]:
test_svd.shape

In [ ]:
trai_svd.drop("Description_clean", axis=1, inplace=True)
test_svd.drop("Description_clean", axis=1, inplace=True)

In [ ]:
def train_model(X=trai_svd, X_test=test_svd, y=y, params=None, folds=folds, model_type='lgb', plot_feature_importance=False,
                averaging='usual', make_oof=False):
    result_dict = {}
    if make_oof:
        oof = np.zeros((len(X), 5))
    prediction = np.zeros((len(X_test), 5))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        gc.collect()
        print('Fold', fold_n + 1, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        if model_type == 'lgb':
            train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols)
            valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols)

            model = lgb.train(params,
                              train_data,
                              num_boost_round=20000,
                              valid_sets=[train_data, valid_data],
                              verbose_eval=500,
                              early_stopping_rounds=200)

            del train_data, valid_data

            y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
            del X_valid
            gc.collect()
            y_pred = model.predict(X_test, num_iteration=model.best_iteration)

        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200,
                              verbose_eval=500, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test), ntree_limit=model.best_ntree_limit)
            
        if model_type == 'lcv':
            model = LogisticRegressionCV(scoring='neg_log_loss', cv=3, multi_class='multinomial')
            model.fit(X_train, y_train)

            y_pred_valid = model.predict_proba(X_valid)
            y_pred = model.predict_proba(X_test)

        if model_type == 'cat':
            model = CatBoostClassifier(iterations=500, loss_function='MultiClass')
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[],
                      use_best_model=False, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict_proba(X_test)

        if make_oof:
            oof[valid_index] = y_pred_valid
            
            
        scores.append(kappa(y_valid, y_pred_valid.argmax(1)))
        print('Fold kappa:', kappa(y_valid, y_pred_valid.argmax(1)))
        print('')

        if averaging == 'usual':
            prediction += y_pred
        elif averaging == 'rank':
            prediction += pd.Series(y_pred).rank().values

        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X.columns
            fold_importance["importance"] = model.feature_importance()
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold

    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))

    if model_type == 'lgb':

        if plot_feature_importance:
            feature_importance["importance"] /= n_fold
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12))
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
            plt.title('LGB Features (avg over folds)')

            result_dict['feature_importance'] = feature_importance

    result_dict['prediction'] = prediction
    if make_oof:
        result_dict['oof'] = oof

    return result_dict


In [ ]:
#########lightgbm###############
print('----------training lightgbm-------------')
params = {'num_leaves': 512,
        #  'min_data_in_leaf': 60,
         'objective': 'multiclass',
         'max_depth': -1,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 3,
         "bagging_fraction": 0.9,
         "bagging_seed": 11,
         "random_state": 42,
         "verbosity": -1,
         "num_class": 5}
result_dict_lgb = train_model(X=trai_svd, X_test=test_svd, y=y, params=params, model_type='lgb', plot_feature_importance=False, make_oof=True)
prediction_lgb = (result_dict_lgb['prediction']).argmax(1)
submission_lgb = pd.DataFrame({'PetID': sub.PetID, 'AdoptionSpeed': [int(i) for i in prediction_lgb]})
submission_lgb.head()
submission_lgb.to_csv('submission.csv', index=False)
print('----------train lightgbm end-------------')


In [ ]:
trai_svd.info()

## retraining on the entire train data

crossvalidation is for model selection only!
Once decided (among types, hyper-params, data pipeline)
→ Use the whole training set to train the selected (type, set of hyperparams and data pipeline)


In [ ]:
#  train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols)
#             valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_cols)

#             model = lgb.train(params,
#                               train_data,
#                               num_boost_round=20000,
#                               valid_sets=[train_data, valid_data],
#                               verbose_eval=500,
#                               early_stopping_rounds=200)

#             del train_data, valid_data

#             y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
#             del X_valid
#             gc.collect()
#             y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
# ##############xgboost##################
# print('----------training xgboost-------------')
# xgb_params = {'eta': 0.01, 'max_depth': 9, 'subsample': 0.9, 'colsample_bytree': 0.9,
#           'objective': 'multi:softprob', 'eval_metric': 'merror', 'silent': True, 'nthread': 4, 'num_class': 5}
# result_dict_xgb = train_model(params=xgb_params, model_type='xgb', make_oof=True)
# prediction_xgb= (result_dict_xgb['prediction']).argmax(1)
# submission_xgb = pd.DataFrame({'PetID': sub.PetID, 'AdoptionSpeed': [int(i) for i in prediction_xgb]})
# submission_xgb.head()
# submission_xgb.to_csv('submission_xgb.csv', index=False)
# print('----------train xgboost end-------------')